In [1]:
!pip install -U sentence-transformers
!pip install jellyfish
!pip install datefinder


from google.colab import drive 
drive.mount('/content/drive') 
%cd /content/drive/My\ Drive/Colab Notebooks/Q&A Bot/Scholarship Q&A

     |████████████████████████████████| 79 kB 5.0 MB/s 
     |████████████████████████████████| 4.0 MB 19.3 MB/s 
     |████████████████████████████████| 1.2 MB 41.9 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 895 kB 54.0 MB/s 
     |████████████████████████████████| 6.5 MB 50.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=da461a884f80f355a5e026f09e47b47bf5f4f5d4cba08a8155a64a6109098bb0
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 132 kB 6.5 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import jellyfish
import datefinder

In [3]:
model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
df = pd.read_csv("final_data.csv")

In [ ]:
questions = df.question.unique()

#GPA

sliced_data = df[df.question == questions[0]]

for val,index in zip(sliced_data.answer.values, sliced_data.index.values):
    score = min(re.findall("\d+\.\d+", val))
    df.at[index,'answer']=score

#SAT

sliced_data = df[df.question == questions[1]]

for val, index in zip(sliced_data.answer.values, sliced_data.index.values):
    scores = re.findall("\d+", val)

    score_sum = 0
    for score in scores:
        score_sum += int(score)

    if 900 < score_sum < 2400:
        df.at[index,'answer']=score_sum
    else:
      df.drop(index, inplace = True)   

# INCOME

sliced_data = df[df.question == questions[5]]

for val,index in zip(sliced_data.answer.values, sliced_data.index.values):
    num = re.findall("\d+\,\d+", val)


    if num: 
        for i,n in enumerate(num):
            num[i] = n.replace(",","")

        money = int(min(num))

    if 20000 < money:
        df.at[index,'answer']=money
    else:
      df.drop(index, inplace = True)   


#AGE

sliced_data = df[df.question == questions[9]]

for val,index in zip(sliced_data.answer.values, sliced_data.index.values):
    ages = re.findall("\d+", val)
    min_age = min(ages, default = "")

    if min_age != "":
        min_age = int(min_age)     
        if 10 < min_age < 40:
            df.at[index,'answer']=min_age
    else:
        df.drop(index, inplace = True)   


#GENDER

sliced_data = df[df.question == questions[10]]

gender = ['female', 'woman', 'girl', 'male', 'man', 'boy'] 

sentence_embeddings = model.encode(sliced_data.answer.values)

similarities = []

for g in gender:
    gender_embedding = model.encode(g)
    similarities.append(cosine_similarity([gender_embedding], sentence_embeddings[:])[0])

similarities = [sum(el) for el in zip(*similarities)]
similarities = [el / len(gender) for el in similarities]


np_similarities = np.array(similarities)
ind = np.where(np_similarities > np_similarities.mean() - np_similarities.var())[0]
#ind = np.where(np.logical_and(np_similarities > np_similarities.mean() + -1*np_similarities.var(), np_similarities < np_similarities.mean() + -0.5*np_similarities.var()))[0]

female_list = []
male_list = []

for ans, index in zip(sliced_data.answer.values[ind], sliced_data.index.values[ind]):

    f_sum = 0
    f_sum2 = 0
    for g in gender[0:3]:
        f_sum += jellyfish.hamming_distance(ans, g)
        f_sum2 = jellyfish.jaro_distance(ans, g)

    m_sum = 0 
    m_sum2 = 0   
    for g in gender[3:6]:
        m_sum += jellyfish.hamming_distance(ans, g)
        m_sum2 = jellyfish.jaro_distance(ans, g)

      
    all_sum = (f_sum + m_sum)
    filter  = np.sqrt((f_sum2 + m_sum2)/len(gender) + np.finfo(float).eps)

    if all_sum > (len(g)*len(gender))/filter or filter < 0.1:
        continue
    elif f_sum < m_sum:
        female_list.append(index)   #change to female
    else:
        male_list.append(index)     #change to male

for index in sliced_data.index.values:
    
    if index in female_list:
        df.at[index,'answer']='F'
    elif index in male_list:
        df.at[index,'answer']='M'
    else:
        df.drop(index, inplace = True)   


#DEADLINE

sliced_data = df[df.question == questions[15]]

for val,index in zip(sliced_data.answer.values, sliced_data.index.values):
    matches = datefinder.find_dates(val)
    
    date = []
    for match in matches:

        date.append(str(match).split()[0])

    if len(date) == 2:
        split_date1 = date[0].split("-")
        split_date2 = date[1].split("-")


        if 0 < int(split_date2[1]) - int(split_date1[1]) < 3:
            final_date = split_date1[0] + "-" + split_date1[1]+ "-" + split_date1[2]
            df.at[index,'answer']=final_date

        else:
            df.drop(index, inplace = True) 
    elif len(date) == 1:

        final_date = date[0]
        df.at[index,'answer']=final_date
    else:
        df.drop(index, inplace = True)  

#ACT

sliced_data = df[df.question == questions[16]]

for val, index in zip(sliced_data.answer.values, sliced_data.index.values):
    score = int(min(re.findall("\d+", val)))

    if 15 < score <= 36:
        df.at[index,'answer']=score
    else:
        df.drop(index, inplace = True)           


In [5]:
questions = df.question.unique()

In [32]:
jellyfish.jaro_winkler("senior","xxxxxx")

0.0

In [40]:
x = []

x.append(5,"x")


TypeError: ignored

In [49]:
sliced_data = df[df.question == questions[3]]

values = sliced_data.answer.values 
indexes = sliced_data.index.values


all_similar_words = {}
for curr_ind,(val,index) in enumerate(zip(values, indexes)):
    similar_word = []
    for val2, index2 in zip(values[curr_ind+1:],indexes[curr_ind+1:]):
        dist = jellyfish.jaro_winkler(val,val2)
        if dist > 0.9:
            word_ind_list = [val2,index2]
            similar_word.append(word_ind_list)

    all_similar_words[index] = similar_word

In [63]:
groups = []
for index, word_list in zip(all_similar_words,all_similar_words.values()):
    for word in word_list:
        word_index = word[1]
        for word_list2 in all_similar_words[word_index]:
            for word2 in word_list2:
                word2_index=word2[1]
            

['high school senior', 1626]


In [48]:
sliced_data.loc[1622].answer

'high school senior'

In [50]:
all_similar_words

{1622: [['high school', 1625],
  ['high school senior', 1626],
  ['high school', 1642],
  ['high school', 1647],
  ['high school', 1650],
  ['high school', 1651],
  ['high school senior', 1652],
  ['high school', 1655],
  ['high school', 1675],
  ['high school', 1678],
  ['high school', 1679],
  ['high school', 1680],
  ['high school', 1683],
  ['high school', 1689],
  ['high school', 1697],
  ['high school', 1700],
  ['high school', 1701],
  ['high school', 1707],
  ['high school', 1711],
  ['high school senior', 1712],
  ['high school', 1717],
  ['high school senior', 1718],
  ['high school senior', 1720],
  ['high school', 1721],
  ['high school', 1725],
  ['high school', 1729],
  ['high school senior', 1730],
  ['high school senior', 1731],
  ['high school', 1734],
  ['high school', 1735],
  ['high school', 1736],
  ['high school', 1737],
  ['high school', 1740],
  ['high school', 1742],
  ['high school', 1745],
  ['high school', 1768],
  ['high school', 1769],
  ['high school', 17

In [ ]:
sliced_data = df[df.question == questions[3]]

for val,index in zip(sliced_data.answer.values, sliced_data.index.values):
    word_dist = []
    for val2,index2 in zip(sliced_data.answer.values, sliced_data.index.values):
        word_dist.append(jellyfish.hamming_distance(val2, val)) 

    #filter
    word_dist_np = np.array(word_dist)
    np.where(word_dist_np < word_dist_np.mean() - word_dist_np.var())[0]

    # check word groups that are inside of the group

    #

    # add word itself
    .append(val)
    len(val)

    all_sum = (f_sum + m_sum)
    filter  = np.sqrt((f_sum2 + m_sum2)/len(gender) + np.finfo(float).eps)

    if all_sum > (len(g)*len(gender))/filter or filter < 0.1:

array(['high school senior', 'undergraduate',
       'junior college, technical school or university', 'high school',
       'high school senior', 'sophomores or juniors',
       'high school senior or first-time college freshman', 'Junior',
       'freshman', 'juniors or seniors', 'junior or senior',
       'junior or senior', 'K-12', 'upper-level', 'fourth year',
       'sophomore', 'sophomore or above', 'undergraduate',
       'third, fourth or fifth-year', 'four-year', 'high school',
       'High school', 'undergraduate', 'four-year',
       'undergraduate, graduate or doctoral', 'high school',
       'undergraduate and graduate', 'two-year or four-year',
       'high school', 'high school', 'high school senior', 'juniors',
       'third, fourth or fifth-year', 'high school',
       'high school grads or GED', 'juniors or seniors',
       'junior or senior', 'High school', 'undergraduate or graduate',
       'undergraduate', '1st year', 'junior', 'sophomore', 'four-year',
       'f

In [6]:
df.question.unique()

array(['What should be the GPA score?', 'What should be the SAT score?',
       'Where should the applicant plan to attend?',
       'Which school year or level should be?',
       'Where should be the residential address or the state of the applicant?',
       'How much should be the total income of the applicant?',
       'Which ethnicity should be?', 'Which race should be?',
       'What nationality or citizenship should the applicant belong to be?',
       'How old should the applicant be?',
       'Which gender should the applicant be?',
       'What religion should the applicant belong to be?',
       'What should be the personal ability or special attribute of the applicant?',
       'What should be the physical disability of the applicant?',
       'What should be the academic major of the applicant?',
       'When is the deadline of the scholarship?',
       'What should be the ACT score?',
       'Which school should the applicant have been accepted into before?'],
      dtyp

In [ ]:
# EDIT CELL

columns = ["GPA", "SAT", "MUST ATTEND INTO", "GRADE", "RESIDENTIAL ADDRESS", "INCOME",
           "ETHNICITY", "RACE", "NATIONALITY", "AGE", "GENDER", "RELIGION", 
           "ABILITY", "DISABILITY", "MAJOR", "DEADLINE", "ACT", "MUST STUDY IN"]
           
quest2col = {}

for question, column in zip(questions, columns):
    quest2col[question] = column 


links = df.link.unique()
questions = df.question.unique()


rows = []

for link in links:
    
    df_slice = df[df.link == link]
    size = df_slice.index.size

    state = df_slice.state.values[0]
    title = df_slice.title.values[0]

    bool_column = {}
    for col in columns:
        bool_column[col] = 0

    qa_dict = {}


    if(size > 0):
        
        for index, row in df_slice.iterrows():
           
            quest = quest2col[row['question']]
            bool_column[quest] = 1

            answer = row['answer']
          
            qa_dict[quest] = answer
    
    for col in columns:
        if bool_column[col] == 0:
            qa_dict[col] = "-"        
    

    qa_dict['TITLE'] = title
    qa_dict['STATE'] = state
    qa_dict['LINK'] = link

    rows.append(qa_dict)   

edit_data = pd.DataFrame(rows)     

In [ ]:
edit_data.to_csv("edit_data.csv", index=False)